In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
import dotenv

dotenv.load_dotenv()


model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

c:\Codes\Learning\Lang_graph\venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [3]:
#state

class LLMState(TypedDict):
    ques: str
    ans: str
    

In [4]:
def llm_question(state: LLMState) -> LLMState:
    #extracting the ques from state
    question = state["ques"]

    #prompt
    prompt = f"Answer the following question: {question}"

    #llm feeding
    response = model.invoke([prompt])

    #updating hte state
    state["ans"] = response.content
    return state

In [6]:
graph = StateGraph(LLMState)

graph.add_node("llm_qa", llm_question)

graph.add_edge(START, "llm_qa")
graph.add_edge("llm_qa", END)

workflow = graph.compile()

In [7]:
#execute
initial_state = {"ques": "What is LangGraph?"}

final_state = workflow.invoke(initial_state)

print(final_state)

{'ques': 'What is LangGraph?', 'ans': 'LangGraph is a Python library designed to make it easier to build **stateful, multi-actor conversational applications** using Large Language Models (LLMs). Think of it as a framework for creating complex, interactive AI agents that can reason, plan, and act over time.\n\nHere\'s a breakdown of its key aspects:\n\n*   **Stateful Conversations:** Unlike simple prompt-response chains, LangGraph allows you to maintain and update the "state" of a conversation over multiple turns. This state can include things like user preferences, intermediate results, or goals the agent is trying to achieve.\n\n*   **Nodes and Edges:** LangGraph represents your application as a graph of interconnected nodes. Each node represents a step in the conversation flow, such as:\n    *   **LLM Calls:** Querying a language model for information or to generate text.\n    *   **Tools:** Using external tools like search engines, calculators, or databases.\n    *   **Conditional L